## 一、检测基座模型的指令遵循能力
对两个数据集，设计两组不同的query，一组有prompt的，一组没有prompt的

In [1]:
import torch
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
import json
import re
import os

# 加载基座模型
path = '/hy-tmp/MiniCPM/finetune/output/results'
input_file_path = '/hy-tmp/cosmosqa-master/data/test.jsonl'

MiniCPM_bf16_path = "/hy-tmp/miniCPM-bf16"
MiniCPM_bf16_tokenizer = AutoTokenizer.from_pretrained(MiniCPM_bf16_path)
MiniCPM_bf16_model = AutoModelForCausalLM.from_pretrained(
    MiniCPM_bf16_path, torch_dtype=torch.float16, device_map="cuda", trust_remote_code=True
)


/usr/local/lib/python3.11/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# 无 prompt
query = "<用户>Text: HGH and steroid use is rampant in track , and , just like in most American professional sports , any incredible individual achievement is questioned immediately , which is a sad state of affairs . The fact that we , as a human population have to assume that someone is using steroids because of an outstanding feat is horrible . What about the people who actually train their hearts out to achieve the same success that the users are trying to achieve ? It is n't fair to the men and women that dedicate their lives to achieving glory the right way , hard work and determination .\n" \
         "Question: Why is HGH and steroid use rampant in track ?\n" \
         "Option 0: Because we have to assume that someone is using steroids because of an outstanding feat .\n" \
         "Option 1: None of the above choices .\n" \
         "Option 2: Because any incredible individual achievement is questioned immediately .\n" \
         "Option 3: Because it 's an American professional sport .\n"

In [5]:
# 有prompt
query_with_prompt = "<用户>You are a reading comprehension expert and below you need to answer multiple choice questions based on the text. Note: Only the number of the option needs to be answered and your answer is only one number.\n" \
         "An example will be given below:\n" \
         "Text: Good Old War and person L : I saw both of these bands Wednesday night , and they both blew me away . seriously . Good Old War is acoustic and makes me smile . I really can not help but be happy when I listen to them ; I think it 's the fact that they seemed so happy themselves when they played .\n" \
         "Question: In the future , will this person go to see other bands play ?\n" \
         "Option 0: None of the above choices .\n" \
         "Option 1: This person likes music and likes to see the show , they will see other bands play .\n" \
         "Option 2: This person only likes Good Old War and Person L , no other bands .\n" \
         "Option 3: Other Bands is not on tour and this person can not see them .\n" \
         "Answer: 1\n\n" \
         "Here are the questions you have to answer: \n" \
         "Text: HGH and steroid use is rampant in track , and , just like in most American professional sports , any incredible individual achievement is questioned immediately , which is a sad state of affairs . The fact that we , as a human population have to assume that someone is using steroids because of an outstanding feat is horrible . What about the people who actually train their hearts out to achieve the same success that the users are trying to achieve ? It is n't fair to the men and women that dedicate their lives to achieving glory the right way , hard work and determination .\n" \
         "Question: Why is HGH and steroid use rampant in track ?\n" \
         "Option 0: Because we have to assume that someone is using steroids because of an outstanding feat .\n" \
         "Option 1: None of the above choices .\n" \
         "Option 2: Because any incredible individual achievement is questioned immediately .\n" \
         "Option 3: Because it 's an American professional sport .\n"

In [6]:
res = MiniCPM_bf16_model.chat(MiniCPM_bf16_tokenizer, query=query, max_length=1024, top_p=0.5)[0]

res_with_prompt = MiniCPM_bf16_model.chat(
    MiniCPM_bf16_tokenizer, query=query_with_prompt, max_length=1024, top_p=0.5)[0]

print(res)
print(res_with_prompt)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Option 2: Because any incredible individual achievement is questioned immediately.
Answer: 2


从上面这个结果可以看出模型具备一定的指令遵循能力

In [12]:
query_tri = "<用户>Question: Who was the man behind The Chipmunks?\n" \
            "Text: Description 1: A struggling songwriter named Dave Seville finds success when he comes across a trio of singing chipmunks: ... Title: Alvin and the Chipmunks (2007) ...\n" \
            "Description 2: The man who brought the Chipmunks to life, ... Five more Chipmunks singles charted in the early '60s, ... See Behind-the-Scenes Rehearsal Photos of Fox's 'The Passion'"

In [26]:
query_tri_with_prompt = "<用户>You are a reading comprehension expert and you need to answer the encyclopaedic questions below based on the text given. Note: Try to be as concise as possible. Do not need to answer in complete sentences.\n" \
            "Example:--Text: Omitted here. --Question: Where in England was Dame Judi Dench born? --Your Answer: York\n" \
            "Here is the question you have to answer:\n" \
            "Question: Who was the man behind The Chipmunks?\n" \
            "Text: Description 1: A struggling songwriter named Dave Seville finds success when he comes across a trio of singing chipmunks: ... Title: Alvin and the Chipmunks (2007) ...\n" \
            "Description 2: The man who brought the Chipmunks to life, ... Five more Chipmunks singles charted in the early '60s, ... See Behind-the-Scenes Rehearsal Photos of Fox's 'The Passion'"

In [22]:
res_tri = MiniCPM_bf16_model.chat(MiniCPM_bf16_tokenizer, query=query_tri, max_length=1024, top_p=0.5)[0]

print(res_tri)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer: Dave Seville


In [27]:
res_tri_with_prompt = MiniCPM_bf16_model.chat(
    MiniCPM_bf16_tokenizer, query=query_tri_with_prompt, max_length=1024, top_p=0.5)[0]

print(res_tri_with_prompt)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Answer: Dave Seville
